# Interacting with Particles in EpiCoMP

In this notebook, we introduce the `particle` object class.  Within the context of this package, a particle is defined as:

**Particle**

> A set of parameters and values used to construct a forward-simulation of a system.

In future notebooks, we will employ a filtering algorithm to 'fit' a set of particles to a desired dataset; however, particles are more generally useful because they encompass the steps and values necessary to simulate a system.

## Example

Let's start out by creating a simple particle object.

**Please note that this notebook assumes that you have properly installed the EpiCoMP package and its dependencies.**


First, we import a 'particle generator'.  This is a factory object - its job is to build us a bunch of particles according to a template that we provide.  For right now, we'll use the default parameter values and only provide the 1 required parameter: N (the number of patients in the population).

Once we've instantiated our generator object, we can use the `.generate()` method to build a new particle according to the template we provided.  Note that you have to provide some kind of ID to the generator - this will allow you to track the history of your particles as you fit them.  While we'll touch on this in a later tutorial, just note that you should really give each of your particles some identifiable name.  Right now, we'll just use '0'.

In [7]:
from covid_particle_filter.particle.SEIHR import SEIHR_generator

In [55]:
example_particle_generator = SEIHR_generator(500000)
example_particle = example_particle_generator.generate(0)

In [56]:
example_particle

Now that we have a particle, let's take a look at what's going on inside that particle.

In [57]:
example_particle.cur_vals

[499950, 50, 0, 0, 0]

In [58]:
example_particle.t

0

In [59]:
example_particle.param_generator.params

{'beta': {'prior_ls': [(<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>,
    7),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 14),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 21),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 28),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 35),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 42),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 49),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 56),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 63),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, None)],
  'cur': None,
  'end': -1},
 'alpha': {'prior_ls': [(<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>,
    None)],
  'cur': None,
  'end': -1},
 'gamma': {'prior_ls': [(<function covid_particle_filter.partic

To start out, our new particle has a set of known current values and a set of Bayesian priors, although no values have been assigned to the parameters yet.  That will happen when the parameters are actually used in our simulation.  Notice that the `beta` parameter has a list of values, each of which ends with a number  (except the last one).  The number listed indicates when the given value should *stop* applying.  So for example, the first generated parameter in that list will be active until time-step 7.  After that, the second value in the list will be generated and become active.  This process prevents the particle filter from filtering out *future* values of a parameter.  So if a particle is re-sampled  on day 3, any new parameters that are generated on day 4 will be generated independently from the same prior distribution.

Let's see how this works by simulating the system forward for a few days.

In [167]:
example_particle_10 = example_particle.copy().step(10, 0, 1)

In [168]:
example_particle.t

0

In [169]:
example_particle_10.t

10.0

In [170]:
example_particle.cur_vals

[499950, 50, 0, 0, 0]

In [171]:
example_particle_10.cur_vals

[499821, 83, 41, 6, 49]

In [172]:
example_particle.param_generator.params

{'beta': {'prior_ls': [(<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>,
    7),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 14),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 21),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 28),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 35),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 42),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 49),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 56),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 63),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, None)],
  'cur': None,
  'end': -1},
 'alpha': {'prior_ls': [(<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>,
    None)],
  'cur': None,
  'end': -1},
 'gamma': {'prior_ls': [(<function covid_particle_filter.partic

In [173]:
example_particle_10.param_generator.params

{'beta': {'prior_ls': [(0.484134718482113, 7),
   (0.5085106588479499, 14),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 21),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 28),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 35),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 42),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 49),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 56),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, 63),
   (<function covid_particle_filter.particle.SEIHR.SEIHR.<lambda>()>, None)],
  'cur': 0.5085106588479499,
  'end': 14},
 'alpha': {'prior_ls': [(0.21111129262581055, None)],
  'cur': 0.21111129262581055,
  'end': None},
 'gamma': {'prior_ls': [(0.18551132339818568, None)],
  'cur': 0.18551132339818568,
  'end': None},
 'h_rate': {'prior_ls': [(0.09342865637160239, None)],
  'cur': 0.093428656371

In [174]:
example_particle.history

In [175]:
example_particle_10.history

array([[499950,     50,      0,      0,      0],
       [499950,     34,     16,      0,      0],
       [499939,     36,     23,      0,      2],
       [499927,     42,     20,      1,     10],
       [499921,     43,     24,      1,     11],
       [499911,     42,     29,      1,     17],
       [499896,     51,     30,      4,     19],
       [499885,     51,     32,      5,     27],
       [499867,     57,     38,      6,     32],
       [499844,     68,     44,      6,     38],
       [499821,     83,     41,      6,     49]])

In [176]:
example_particle_10.particle_id

'00'

Here, we see that our simulation has moved forward 10 days, with individual time-steps on every day between.  We can track the movements between compartments over time using the `.history` value.

The `.step()` command initiates a forward simulation.  It requires 3 input parameters.  First, the target timestamp (in our case, 10).  Second, it requires a new ID to be added to the old one (in our case, 0 - thus we get the new ID '00').  Finally, it requires your timestep (in our case, 1).  A smaller timestamp will make the system more reliable, but it can also take longer to run.  Note that the system will only remember values at integral timestamps (0, 1, 2, etc.) and will not accept timesteps greater than the distance to your desired new timestamp (e.g. a target time of 1 with a timestep of 2).  

The minimum 'reporting' step size (i.e. the size of the step between values recorded in the `_t_history` and `history` values) is 1.  Anything smaller than that will be aggregated together to  approximate a minimum step size of 1 (although this can be imperfect, as below).

If your timestep does not add to your target timestamp evenly, the system will force an additional break to ensure that the simulation completes on the target timestamp.  This can slightly alter the step size, as below.  **We do not recommend using large timesteps, as this will significantly degrade simulation quality.**

**In general, we recommend using a step size that evenly divides 1, typically either 0.1 or 0.01.**

Note that we used the method `example_particle.copy().step()`, not `example_particle.step()`.  Both of these methods are completely valid, and both would return a new object that needs to be assigned to a variable.  The difference is that the `.copy()` command prevents the original `example_particle` from 'remembering' the parameters generated from its priors.  If we leave that command out, we'd still have an `example_particle` object with a current time-stamp of 0, but all parameters generated during the forward simulation would be retained.  This could be useful if you want to simulate a given set of parameter values many times.

In [89]:
example_particle.copy().step(10,0,0.1)._t_history

[0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

In [94]:
example_particle.copy().step(10,0,0.15)._t_history

[0,
 1.0606060606060606,
 2.121212121212121,
 3.0303030303030303,
 4.090909090909091,
 5.0,
 6.0606060606060606,
 7.121212121212121,
 8.030303030303031,
 9.090909090909092,
 10.0]

In [90]:
example_particle.copy().step(10,0,1.4)._t_history

[0,
 1.4285714285714286,
 2.857142857142857,
 4.285714285714286,
 5.714285714285714,
 7.142857142857143,
 8.571428571428571,
 10.0]

In [88]:
example_particle.copy().step(10,0,2)._t_history

[0, 2.0, 4.0, 6.0, 8.0, 10.0]

## Under the Hood

Behind the scenes, we're using a simple function to calculate each step forward.  In the absence of a target value to evaluate, we can reduce this to an iterative version of a single function.

In [96]:
from covid_particle_filter.particle.SEIHR import SEIHR_update

The `SEIHR_update` function performs a simple step-forward over a single time-step.  While we have provided step-functions for the SEIHR and SEIQHR functions, it is possible to implement alternative models within our framework.  To do so would primarily require the implementation of a new update function, such as this one.

The update function takes three parameters.  The `vals` parameter takes the current values of the system variables; the `params` parameter takes the current values of relevant system parameters; and the `dt` function takes the size of your timestep.

For ease of discussion and understanding, we have recreated the code (without documentation) below.  Please note that our new `SEIHR_update_nonimport` is identical to the original version.

In [100]:
import numpy as np

def SEIHR_update_nonimport(vals, params, dt):
    
    S, E, I, H, R = vals
    beta, alpha, gamma, h_rate = params
    
    N = np.sum(vals)
    
    p_ls = [
            beta*I/N*dt,
            alpha*dt,
            gamma*dt
        ]
    
    n_ls = [
        S,
        E,
        I
    ]
    
    new_e, new_i, i_out = np.random.binomial(n_ls, p_ls)
    new_h = np.random.binomial(i_out, h_rate)
    new_r = i_out - new_h
    
    S = S - new_e
    E = E + new_e - new_i
    I = I + new_i - i_out
    H = H + new_h
    R = R + new_r
    
    return [S, E, I, H, R]

The core idea of this function is that we use `Numpy` to generate an array of the number of patients transitioning from each compartment to the next.  Our implementation uses a Binomial distribution to accomplish this.  Essentially, each patient is assigned an independent probability of conversion based on the underlying conversion rates (`alpha`, `beta*I/N`, and `gamma`).  Then, we multiply these probabilities by the step-size, because these probabilities are assumed to be averaged over a step-size of 1.

This process yields a vector indicating the number of patients transitioning out of each compartment, which we add or subtract (as appropriate) to compute the new state.  We also split the stream of patients exiting the `Q` compartment based on the `h_rate` parameter using a secondary Binomial sample.

## Creating Custom Prior Distributions

Now that we understand some of the basic requirements for interacting with a particle, let's discuss how we can customize a particle.  For now, we'll assume that you need to customize a particle to use your own prior distribution, but that you don't need to create a new compartment model entirely.

To make this process as simple as possible, the custom parameter generation objects used by the simulation are all obfuscated.  You only need to provide a dictionary of parameter values or generators which follows a simple pattern.

#### Parameter Dictionary Rules

1. You must specify a prior for every relevant parameter.  If you don't specify a parameter, your call to the `.generate()` command will fail.
2. Priors for parameters must be specified as a list of tuples.  Each tuple should include (1) the desired value or generator (which should return a value from simple evaluation call) and (2) a target expiration date for the parameter.  A `None` value indicates that this parameter should never expire.  Note that the last value in the list will be replaced with a `None` if one is not provided.

In [185]:
custom_SEIHR_param_prior = {
    'beta'  : [(4, 7),
               (lambda : np.random.gamma(2/.525, .075), None)],
    'alpha' : [(lambda : np.random.gamma(0.3/0.02, 0.02), None)],
    'gamma' : [(lambda : np.random.gamma(0.15/0.01, 0.01), None)],
    'h_rate' : [(lambda : gen_h_rate(), None)]
}

In [112]:
example_particle_generator_param = SEIHR_generator(500000, param_prior_dict = custom_SEIHR_param_prior)
example_particle_generator_param.generate(0).param_generator.params

{'beta': {'prior_ls': [(4, 7), (<function __main__.<lambda>()>, None)],
  'cur': None,
  'end': -1},
 'alpha': {'prior_ls': [(<function __main__.<lambda>()>, None)],
  'cur': None,
  'end': -1},
 'gamma': {'prior_ls': [(<function __main__.<lambda>()>, None)],
  'cur': None,
  'end': -1},
 'h_rate': {'prior_ls': [(<function __main__.<lambda>()>, None)],
  'cur': None,
  'end': -1}}

#### Initial Value Dictionary Rules

1. You must specify a prior for each of the values.
2. Prior values will be normalized, so your values do not need to add to any particular value.
3. Values should be specified using a similar syntax to the parameter list above, but no tuples or expiration dates are necessary.
4. Generated initial values will always be rounded to integers.

In [118]:
custom_SEIHR_init_prior = {
        'S'   : 10000,
        'E'   : 100,
        'I'   : lambda : (np.random.poisson(9)+1),
        'H'  : np.random.normal(100,10),
        'R'   : 0,
    }

In [119]:
example_particle_generator_init = SEIHR_generator(500000, init_prior_dict = custom_SEIHR_init_prior)
example_particle_generator_init.generate(0).cur_vals

[489703, 4897, 391, 5009, 0]

# Particle Evaluation

As particles simulate a system, we want to be able to use an evaluation metric to assign each particle a weight.  These weights can be used by the Particle Filter to decide which particles to eliminate or genetically recombine.  The `_evaluate` method of the particle class performs this task.  This method takes the target observation (e.g. hospital admissions over the evaluation period, typically one day), the current state of the system, and the values that were active at the beginning of the evaluation period (e.g. one day ago).  Then, the method outputs a particle weight, which is equal to the previous particle weight multiplied by the new probability of obtaining the current data (weights are typically re-normalized afterwards relative to all other particles to prevent numeric overflow errors).

As an example, we present the code for the SEIHR evaluate method below.

In [127]:
import pandas as pd
import numpy as np
from scipy.stats import poisson, binom

def _evaluate(self, eval_value, cur_vals, prev_vals):
    #number of hospital admissions observed
    delta_H_plus = eval_value
    
    #total number of patients transitioning to either H or R
    delta_plus_sim = np.sum(np.array(cur_vals)[[3,4]]) - np.sum(np.array(prev_vals)[[3,4]])

    #for the purpose of our evaluation here, the update weight is the probability of obtaining the observed hospital admissions
    #by sampling from a binomial with h_rate success probability and delta_r+delta_h samples
    prob_plus = binom.pmf(delta_H_plus, delta_plus_sim, self.param_generator.params['h_rate']['cur'])

    #the new weight is multiplied by the old rate; we add a small constant to prevent multiplication by zero
    w_t = self.w_t * (prob_plus+10**(-4))
    return w_t

When we review particle fitting, we'll see how this function is used.  However, if you need to respecify this metric while retaining the rest of the particle object's methods, you can sub-type your own version of the SEIHR particle, like below.

In [131]:
from covid_particle_filter.particle.SEIHR import SEIHR_Particle

import pandas as pd
import numpy as np
from scipy.stats import poisson, binom

class my_custom_SEIHR(SEIHR_Particle):
    def _evaluate(self, eval_value, cur_vals, prev_vals):
        """This is our custom evaluate function that we can specify however we like."""
        #For simplicity, just return 1
        return 1

# Hospital Census

Thus far, we have focused exclusively on hospital admissions.  For many, this application will be sufficient. Furthermore, the material thus far is all that is necessary to _fit_ a particle.  However, some forecasts may require estimates of daily hospital census.  To facilitate this, we have constructed a separate set of objects, which allow the user to fit a hospital compartment separately (based on the observed hospital-to-discharge times, using KM survival estimation function).  These functions fit entirely independently from the particle filter, but can be combined with the SEIHR or SEIQHR particles to construct simulations with estimates of instantaneous hospitalization rates.

In [135]:
from covid_particle_filter.particle.HCompartment import HCompartment

In [136]:
h = HCompartment()

To fit the Hospitalization compartment, we need two one-dimensional arays.  The first should contain the lengths of stay for patients admitted to the hospital and the second should contain True/False indicators of whether an observation was censored (i.e. the patient is still in the hospital).

Internally, this function is using the [LifeLines Kaplan-Meier Fitter](https://lifelines.readthedocs.io/en/latest/fitters/univariate/KaplanMeierFitter.html) to construct a model of hospital length-of-stay.

In [142]:
los_ls = np.array([1,3,1,0,5,10])
cens_ls = np.array([False]*2 + [True]*3 + [False])

In [143]:
h.fit(los_ls, cens_ls)

C:\Users\P030307\AppData\Local\Continuum\anaconda3\lib\site-packages\lifelines\fitters\kaplan_meier_fitter.py:346: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  self.confidence_interval_ = self._bounds(cumulative_sq_[:, None], alpha, ci_labels)


Once we've fit our HCompartment, we can generate a series of simulated stays based on the fitted distribution.

The `generate` method takes as inputs `N`, the number of stays to generate and `conditional`, which is a numeric indicating that the stay-lengths generated must be larger than the given number.  This is useful if we want to simulate when a series of current ongoing stays will end.

In [146]:
h.generate(10, 2)

array([10.25863363,  9.9207958 ,  6.01689189,  9.38588589,  7.91253754,
        7.55593093,  3.53303303,  6.72072072, 11.        ,  2.91741742])

The greatest advantage to using the `HCompartment` object is that it allows us to compute instantaneous hospital census, now and in the future.  To do this, we'll need to combine the `HCompartment` with the instantaneous `H` compartment in our particles (above).  The `generate` function above allows us to simulate future hospitalizations, but we still need to add historical discharge dates.  The `add_history` function gives us the option to do this in bulk.

In [147]:
h.add_history([3]*3+[4]+[10]*5)

The `get_history` function gives us the number of cumulative discharges observed by the given timepoint.  If we provide multiple times (in an array), we get back an array with the cumulative discharges by the provided time.

In [150]:
h.get_history(np.linspace(0,10,11))

array([0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 9], dtype=int64)

In addition to adding a history, we can `update` our hospital compartment with new hospital admissions.  This is useful for forward simulation.  Our inputs include:

    in_ (int/iterable of ints): the number of discharges to simulate
    t (numeric/iterable of numerics): the admission times of the discharges to simulate
    conditional (numeric/iterable of numerics/None): the conditionals of the discharges to simulate

This function only updates the history and provides no outputs.

In [151]:
h.update([10, 5], [0, 4], None)

In [152]:
h.get_history(np.linspace(0,10,11))

array([ 0,  0,  2,  6,  7,  7,  9, 12, 14, 15, 20], dtype=int64)

Note that we can also use the `clear()` command to clear our history and the `copy()` command to create copies that will not be mutable (important when reusing a trained hospital compartment across simulations).

# Combined Particles

Now, we want to combine the particles before with our new `HCompartment` objects.  All interactions between the two can be handled with a `SEIHR_combined_particle` object.  This object requires that we supply:

    particle (SEIHR_Particle): A particle fit using the particle filter in this package
    h_compartment (HCompartment): An HCompartment object fit using this package
    admits (int): The total number of admissions to-date
    discharges (int): The total number of discharges to-date
    open_admit_dts (list): A list of the dates of admissions without corresponding discharges
    
For now, we'll create a simulation which assumes that our system has had a total of 10 admissions to-date and where 8 have already been discharged (current hospital census is 2).

In [177]:
from covid_particle_filter.particle.SEIHR import SEIHR_combined_particle

In [178]:
cpart = SEIHR_combined_particle(example_particle_10, h, 10, 8, [8,9])

This final particle can run combined forecasts with _six_ compartments.  It replaces the fourth compartment (H) with a new instantaneous version of the compartment which tracks how many patients are currently in the hospital (essentially the number of admissions less the number of discharges).  Then, it adds another compartment at the end (position 6 here) to indicate the total number of discharges to-date.

Now, we run a forecast until timestamp 20 and look at the results (note that the first set of values corresponds to our presumed current values).

In [181]:
cpart.run_forecast(20, 0.1)

In [183]:
cpart.full_forecast

array([[499821,     83,     41,      2,     45,      8],
       [499795,     90,     51,      3,     53,      8],
       [499765,    101,     61,      3,     62,      8],
       [499729,    111,     79,      2,     70,      9],
       [499679,    133,     92,      3,     84,      9],
       [499650,    132,    103,      4,    101,     10],
       [499608,    146,    111,      7,    118,     10],
       [499575,    147,    123,      9,    136,     10],
       [499532,    159,    133,      8,    157,     11],
       [499490,    167,    147,      9,    176,     11],
       [499448,    170,    151,     12,    208,     11]], dtype=int64)

Internally, the Combined particle actually uses three slightly different versions of the same particle.  The historical particle stores your history - you provide it.  The base future particle acts as a trimmed-down base for future simulation, erasing all history until the current values (for efficiency), but retaining all of the parameters and current values used to simulate forward.  The future particle is generated when we use  the `run_forecast` method (and  gets replaced each time we run the forecast).  Note that there are some functions (such as `replace_prior`) which target the `base_fut_particle` and leave the fitted particle untouched.